In [109]:
import pandas as pd
file_path = 'clustering-tendency.csv'
df = pd.read_csv(file_path, header=None)
df

,0,1,2
0,0.964680,0.351274,0.298323
1,0.585918,0.800970,0.187782
2,0.509113,0.280177,0.208792
3,0.352107,0.665974,0.046316
4,0.363669,0.323479,0.219902
...,...,...,...
995,0.592656,0.516658,0.699137
996,0.083745,0.542244,0.644479
997,0.997039,0.101037,0.305323
998,0.818407,0.608719,0.391878


In [110]:
df.apply(lambda x : x[0] > 0.75 and x[1] > 0.75 and x[2] > 0.75, axis=1).value_counts(sort=False)

False    1000
Name: count, dtype: int64

In [111]:
from math import floor
def categorize(data_point):
    return (floor(data_point[0] / 0.25)) * 16 + (floor(data_point[1] / 0.25)) * 4 + (floor(data_point[2] / 0.25))



categories_3d =  df.apply(categorize, axis="columns")
categories_3d

0      53
1      44
2      36
3      24
4      20
       ..
995    42
996    10
997    49
998    57
999     9
Length: 1000, dtype: int64

In [112]:
from math import log

rel_prob_3d:pd.Series = categories_3d.value_counts(normalize=True)
rel_prob_3d

16    0.050
17    0.037
33    0.036
1     0.036
20    0.033
52    0.032
48    0.029
53    0.028
21    0.028
32    0.028
37    0.028
36    0.027
5     0.026
49    0.025
44    0.025
0     0.022
8     0.022
56    0.021
4     0.021
12    0.021
27    0.020
6     0.019
26    0.019
40    0.018
23    0.017
25    0.017
28    0.016
57    0.016
45    0.016
43    0.016
61    0.016
42    0.016
29    0.016
9     0.015
58    0.015
60    0.015
54    0.015
41    0.015
11    0.014
24    0.014
10    0.014
38    0.013
59    0.013
22    0.012
13    0.011
55    0.011
39    0.010
7     0.010
2     0.002
46    0.001
51    0.001
47    0.001
18    0.001
Name: proportion, dtype: float64

In [113]:
test = pd.Series(rel_prob_3d, index=categories_3d.index)


def calc_entropy(probs: pd.Series):
    sum = 0
    for i in range(0, 63):
        value = probs.get(i, default=0)
        if 0 < value < 1:
            sum += (value * log(value) +
                    (1 - value) * log(1 - value))
    return sum * -1


calc_entropy(rel_prob_3d)

4.8075123183976665

In [114]:
import itertools

def calculate_2d(data_point: pd.Series):
    return (floor(data_point.values[0] / 0.125)) * 8 + (floor(data_point.values[1] / 0.125))

for pair in itertools.combinations(df.columns, r=2 ):
    data = df[list(pair)]
    bined_data = data.apply(calculate_2d, axis=1).value_counts(normalize=True)
    print(f"{pair} : {calc_entropy(bined_data)}")

(0, 1) : 4.9775863992866896
(0, 2) : 4.758252517308642
(1, 2) : 4.392500622437077


In [115]:
def calculate_1d(data_point: float, bucket_number: int = 64):
    return floor(data_point / (1/bucket_number))


def process_column_1d(column: int):
    global binned_data, count_data, entropy
    print("Column ", column)
    binned_data = df[column].apply(calculate_1d)
    count_data = binned_data.value_counts(normalize=True)
    entropy = calc_entropy(count_data)
    print(f"{column} : {entropy}")
    return binned_data, count_data, entropy

for column in df.columns:
    process_column_1d(column)

Column  0
0 : 5.009714758101686
Column  1
1 : 4.968396695785046
Column  2
2 : 4.702932411963032
